# Visualise results from model trained in clustering_analysis_training.ipynb

Author: Katharine Leney, April 2025

In [1]:
import joblib
import pandas as pd
import pickle
import os
import json
import sys
import plotly.express as px  
sys.path.append("../src")
from utils.parse_model_folder import parse_model_folder
from utils.generate_clean_labels import generate_clean_labels
from utils.generate_opportunity_table import generate_opportunity_table

In [2]:
# ====================================
# Load model and associated data
#
# !! Set path to model you want to visualise here !!
#
# ====================================

# Model trained on full dataset
#model_name = f"bertopic_cluster_model__20250428"

# Model trained on 2015--2019, 2022--2024 data
model_name = f"bertopic_cluster_model_from_2015_excl_2020_2021_20250428"

# Build correct path
model_path = os.path.join("../models/", model_name, "model")

# Load the model
topic_model = joblib.load(model_path)

# Load texts and timestamps
with open(os.path.join("../models/", model_name, "texts.pkl"), "rb") as f:
    texts = pickle.load(f)

with open(os.path.join("../models/", model_name, "timestamps.pkl"), "rb") as f:
    timestamps = pickle.load(f)

model_label = parse_model_folder(model_name)
print(model_label)

# Make sure the outputs folder exists
os.makedirs("../outputs", exist_ok=True)

/Users/katharine/anaconda3/envs/iata/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From 2015 | Excluding 2020, 2021


In [3]:
# Explore top topics
topic_model.get_topic_info().head(10)
#topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,148,-1_airlines_chairman_president_fuel,"[airlines, chairman, president, fuel, mc99, ti...",[7 CHAIRMAN CALIN ROVINESCU CHAIR OF THE BOARD...
1,0,166,0_passenger_cargo_growth_year,"[passenger, cargo, growth, year, load, domesti...",[Airline profitability per passenger 85 83 81 ...
2,1,148,1_saf_emissions_carbon_corsia,"[saf, emissions, carbon, corsia, environmental...",[ENVIRONMENT & SUSTAINABILITY The journey to n...
3,2,129,2_safety_accidents_iosa_accident,"[safety, accidents, iosa, accident, audit, air...",[Safety IATA Annual Review 2022 20 All acciden...
4,3,109,3_airport_charges_infrastructure_airports,"[airport, charges, infrastructure, airports, i...",[But attempts to reduce significant increases ...
5,4,105,4_travel_ndc_order_experience,"[travel, ndc, order, experience, passenger, pa...",[With passenger volume expected to double by 2...
6,5,101,5_security_information_smart security_smart,"[security, information, smart security, smart,...",[Smart Security Smart Security is joint initia...
7,6,95,6_cargo_ceiv_data_lithium,"[cargo, ceiv, data, lithium, goods, chain, fre...",[Special cargo Standardized air cargo processe...
8,7,82,7_world_training_itdi_commercial,"[world, training, itdi, commercial, strategic,...",[World Cargo Symposium Legal Symposium Ops Con...
9,8,63,8_settlement_billion_processed_iss,"[settlement, billion, processed, iss, bsp, fin...","[It operates through CASSlink, an advanced, gl..."


In [4]:
# ------------------------------------------------------------
# Visualize intertopic distances
# ------------------------------------------------------------

# N.B. UMAP (Uniform Manifold Approximation and Projection)
# used for dimensionality reduction. 
fig_2d = topic_model.visualize_topics(custom_labels=True)

# Update background and grid
fig_2d.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    font_family="Arial",
    title={
        "text": f"Intertopic Distance Map<br><sup>{model_label}</sup>",
        "x": 0.15,
        "font": {"size": 20}
    },
    width=1000,
    height=800,
    xaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=True,
        ticks="outside",
        title=None
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)
fig_2d.write_html("../outputs/intertopic_distance_map_2D.html")   

fig_2d.show()

In [5]:
# ------------------------------------------------------
# Clean up labels and remove junk topics
# ------------------------------------------------------

# Retrieve the current model labels before overwriting
old_labels = topic_model.topic_labels_

# Generate new labels
clean_labels, junk_topics = generate_clean_labels(topic_model, top_n_words=1)

# Print label changes for review
print("\nGenerated topic labels:")
for topic_id, label in sorted(clean_labels.items()):
    old_label = old_labels.get(topic_id)
    print(f"Topic {topic_id}: {label} (OLD: {old_label})")

# Apply to the model
topic_model.set_topic_labels(clean_labels)

Topic  15 identified as junk:
[('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]

Generated topic labels:
Topic -1: NULL (OLD: -1_airlines_chairman_president_fuel)
Topic 0: Passenger (OLD: 0_passenger_cargo_growth_year)
Topic 1: SAF (OLD: 1_saf_emissions_carbon_corsia)
Topic 2: Safety (OLD: 2_safety_accidents_iosa_accident)
Topic 3: Airport (OLD: 3_airport_charges_infrastructure_airports)
Topic 4: Travel (OLD: 4_travel_ndc_order_experience)
Topic 5: Security (OLD: 5_security_information_smart security_smart)
Topic 6: Cargo (OLD: 6_cargo_ceiv_data_lithium)
Topic 7: World (OLD: 7_world_training_itdi_commercial)
Topic 8: Settlement (OLD: 8_settlement_billion_processed_iss)
Topic 9: Tax (OLD: 9_tax_taxation_taxes_government)
Topic 10: Airlines (OLD: 10_airlines_airways_china_latam)
Topic 11: Regulation (OLD: 11_regulation_smarter_regulations_smarter regulation)
Topic 12: Slot (OLD: 12_slot_wsg_slots_airport)
To

In [6]:
# ------------------------------------------------------------
# Generate topic time series and visualize trends
# ------------------------------------------------------------

timestamps = pd.to_datetime(timestamps, format="%Y")
topics_over_time = topic_model.topics_over_time(texts, timestamps, topic_model.topics_)

# Save time series data and topic labels for the RAG assistant to use later
topics_over_time.to_csv("../outputs/topics_over_time.csv", index=False)
with open("../outputs/topic_labels.json", "w") as f:
    json.dump(clean_labels, f, indent=2)

# Filter out junk and noise topics from list and count rate
topic_freq = topic_model.get_topic_freq()
valid_topic_freq = topic_freq[
    (~topic_freq["Topic"].isin(junk_topics)) &
    (topic_freq["Topic"] != -1)
]

# Select top k most frequent clean topics
top_k = 10
top_topics = valid_topic_freq.head(top_k)["Topic"].tolist()

# Make plot of topic frequency by year
# y-axis = % of all text chunks assigned to this topic in a given year
fig = topic_model.visualize_topics_over_time(topics_over_time, topics=top_topics)

# Apply custom colors and labels
palette = px.colors.qualitative.T10

for i, trace in enumerate(fig.data):
    topic_id = top_topics[i]
    trace.line.color    = palette[i % len(palette)]
    trace.name = clean_labels[topic_id]
    hover_text = trace.hovertemplate or ""
    trace.hovertemplate = f"{clean_labels[topic_id]}<extra></extra>"

# Clean up the figure layout
fig.update_layout(
    title=f"Topic Trends Over Time<br><sup>{model_label}</sup>",
    width=1200,
    height=600,
    xaxis_title="Year",
    yaxis_title="Topic Frequency (% of Annual Report)",
    legend_title="Topics",
    font=dict(family="Arial", size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
    )
)

output_filename = f"../outputs/topic_trends_{model_name}"
fig.write_html(f"{output_filename}.html")
fig.write_image(f"{output_filename}.png")

fig.show()

8it [00:01,  7.90it/s]


In [7]:
# ===============================================
# Generate a table matching trends to business
# opportunities, and assign a priority ranking
# based on trend size and growth
# ===============================================

#topics_over_time = topic_model.topics_over_time(texts, timestamps, topic_model.topics_)
matched_opportunities_df, unmatched_topics = generate_opportunity_table(topic_model, topics_over_time)

print(f"Filtering (if any): {model_label}")
print(f"Model Name: {model_name}\n")
print("=== Matched Business Opportunities ===\n")
display(matched_opportunities_df)

# Save table as CSV
output_filename = f"../outputs/business_opportunities_{model_name}.csv"
matched_opportunities_df.to_csv(output_filename, index=False)

print(f"\nSaved opportunity table to: {output_filename}")

Filtering (if any): From 2015 | Excluding 2020, 2021
Model Name: bertopic_cluster_model_from_2015_excl_2020_2021_20250428

=== Matched Business Opportunities ===



,Trend,Detected Label,Implication,Opportunity,Priority (Recommended),Keyword Fraction (%),Topic Growth
0,Growth,0_passenger_cargo_growth_year,Strong recovery in air travel post-pandemic,Expand network capacity and optimize route pla...,High,13.47,7
1,Emissions,1_saf_emissions_carbon_corsia,Increasing pressure for environmental responsi...,Adopt Sustainable Aviation Fuels (SAF) and car...,High,12.01,13
2,Safety,2_safety_accidents_iosa_accident,Continued industry focus on safety standards,Invest in IOSA certification and safety audits,Medium,10.47,-4
3,Airport,3_airport_charges_infrastructure_airports,Growing airport-related fees and costs,Negotiate airport charges and improve operatio...,Medium,8.85,0
4,Security,5_security_information_smart security_smart,Passenger data security and travel security co...,Enhance digital identity management and passen...,Medium,8.20,1
5,Cargo,6_cargo_ceiv_data_lithium,Continued strength in air cargo and freight de...,Expand cargo services and invest in logistics ...,Medium,7.71,-4
6,Training,7_world_training_itdi_commercial,Ongoing need for aviation training and upskilling,Expand training programs and digital learning ...,Medium,6.66,-24
7,Settlement,8_settlement_billion_processed_iss,"Changes in financial settlement systems (BSP, ...",Optimize settlement systems and implement fast...,Medium,5.11,-2
8,Airport,12_slot_wsg_slots_airport,Growing airport-related fees and costs,Negotiate airport charges and improve operatio...,Medium,2.44,3
9,ID,13_unruly_mp14_incidents_passengers,Move towards digital identity and seamless travel,Implement biometric and digital ID solutions,Medium,2.27,4



Saved opportunity table to: ../outputs/business_opportunities_bertopic_cluster_model_from_2015_excl_2020_2021_20250428.csv
